In [1]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT.name != "Thesis code" and PROJECT_ROOT != PROJECT_ROOT.parent:
    PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))

from Functions.data_utils import (
    plot_incremental_response_rate,
    uplift_by_decile_bin,
    coerce_metrics_to_numeric,
    add_uplifts,
    uplift_mmoa
)



C:\Users\tsterk\AppData\Local\anaconda3\envs\causalml_py312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to import duecredit due to No module named 'duecredit'


In [2]:
import pandas as pd

pd.set_option("display.max_columns", None)


In [3]:
from __future__ import annotations

import time
from typing import Tuple, Union

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.calibration import CalibratedClassifierCV
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    StratifiedShuffleSplit,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import resample
from sklearn.base import clone

from catboost import CatBoostClassifier

from classifier_calibration.calibration_error import classwise_ece

In [4]:
file_path = r"Data/covariates_modeling_uplift_models_2026-02-28.csv"
df = pd.read_csv(file_path)


C:\Users\tsterk\AppData\Local\Temp\ipykernel_3912\4048377555.py:2: DtypeWarning: Columns (0: total_volume, 1: food_total, 2: sports_total, 3: beauty_total, 4: monetary_value_52wk, 5: online_sales_52w, 6: retail_sales_52w, 7: monetary_value_53w_104w, 8: online_sales_53w_104w, 9: retail_sales_53w_104w) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [5]:
df['cameback'].value_counts()

cameback
0    264346
1      3321
Name: count, dtype: int64

In [6]:
treatment_converter = {
    "BNLX_ChurnP_10_test_export.csv": 1,
    "BNLX_ChurnP_10_controle_export.csv": 0, 
    
    "BNLX_ChurnP_25_test_export.csv": 2,
    "BNLX_ChurnP_25_controle_export.csv": 0,

    "BNLX_ChurnP_5eu_test_export.csv": 3,
    "BNLX_ChurnP_5eu_controle_export.csv": 0,

    "BNLX_ChurnP_10eu_test_export.csv": 4,
    "BNLX_ChurnP_10eu_controle_export.csv": 0,
    
    "BNLX_ChurnP_250_test_export.csv": 5,
    "BNLX_ChurnP_250_controle_export.csv": 0,

    "BNLX_ChurnP_500_test_export.csv": 6,
    "BNLX_ChurnP_500_controle_export.csv": 0,    
    
    "BNLX_ChurnP_SKUd_test_export.csv": 7,
    "BNLX_ChurnP_SKUd_controle_export.csv": -99,

    "BNLX_ChurnP_SKUe_test_export.csv": 8,
    "BNLX_ChurnP_SKUe_controle_export.csv": 0,    
    
    "BNLX_ChurnP_niks_test_export.csv": 9,
    "BNLX_ChurnP_niks_controle_export.csv": 0,
}

df["treatment"] = df["treatment_indicator"].map(treatment_converter)


In [7]:
categorical_cols  = ['has_rfl','gender','country_sk']
numeric_cols = [ 'recency', 'frequency', 'monetary_value',  'total_volume', 'length_of_relationship', 'online_sales', 'retail_sales', 
      'food_total', 'vhms_total', 'sports_total', 'beauty_total', 'frequency_52wk', 'monetary_value_52wk', 'volume_52wk', 'online_sales_52w', 
       'retail_sales_52w', 'frequency_53w_104w', 'monetary_value_53w_104w', 'volume_53w_104w', 'online_sales_53w_104w','retail_sales_53w_104w']

In [8]:
df = coerce_metrics_to_numeric(df, numeric_cols)

df[categorical_cols] = df[categorical_cols].astype("object")
df[numeric_cols] = df[numeric_cols].astype("int64")


In [9]:
(100 * df.groupby('treatment_indicator')['cameback'].mean()).round(2)

treatment_indicator
BNLX_ChurnP_10_controle_export.csv      1.08
BNLX_ChurnP_10_test_export.csv          1.25
BNLX_ChurnP_10eu_controle_export.csv    1.25
BNLX_ChurnP_10eu_test_export.csv        1.45
BNLX_ChurnP_250_controle_export.csv     1.13
BNLX_ChurnP_250_test_export.csv         1.46
BNLX_ChurnP_25_controle_export.csv      1.13
BNLX_ChurnP_25_test_export.csv          1.24
BNLX_ChurnP_500_controle_export.csv     1.27
BNLX_ChurnP_500_test_export.csv         1.53
BNLX_ChurnP_5eu_controle_export.csv     1.08
BNLX_ChurnP_5eu_test_export.csv         1.37
BNLX_ChurnP_SKUd_controle_export.csv    1.12
BNLX_ChurnP_SKUd_test_export.csv        1.13
BNLX_ChurnP_SKUe_controle_export.csv    1.17
BNLX_ChurnP_SKUe_test_export.csv        1.25
BNLX_ChurnP_niks_controle_export.csv    1.15
BNLX_ChurnP_niks_test_export.csv        1.24
Name: cameback, dtype: float64

In [10]:
df["multi_outcome"] = np.where(
    df["cameback"].to_numpy() != 0,
    "cameback_" + df["treatment"].astype(str),
    "no_cameback_" + df["treatment"].astype(str),
)

df = df[df['multi_outcome'].isin(['cameback_9','no_cameback_9','cameback_8','no_cameback_8',
                                'cameback_6','no_cameback_6','cameback_5','no_cameback_5',
                                  'cameback_4','no_cameback_4','cameback_3','no_cameback_3',
                                  'cameback_2','no_cameback_2','cameback_1','no_cameback_1',
                                  'cameback_0','no_cameback_0'])]

# 3) Set y to multi_outcome, exclude treatment and visit from X
target_col = "multi_outcome"

X = df.drop(columns=[target_col, "treatment_indicator", 'treatment', "cameback", 'customer_nk'])
y = df[target_col]


In [11]:
################################################################################################
# Function to return the prior probabilities of treatments per treatment group, i.e. 0 = control
# Used for counteracting the imbalance of treatment groups
################################################################################################

def get_treatment_probs_from_y_true(
    df: pd.DataFrame,
    *,
    y_true_col: str = "y_true",
) -> Dict[int, float]:
    """
    Compute P(T=t) using only the last digit of y_true (e.g., 'cameback_3' -> 3).
    Returns a dict like {0: 0.52, 1: 0.11, ...}.
    """
    probs = (
        df[y_true_col]
        .astype(str)
        .str[-1]
        .astype(int)
        .value_counts(normalize=True)
        .sort_index()
        .to_dict()
    )

    return probs

treatment_probs = get_treatment_probs_from_y_true(df, y_true_col='multi_outcome')
treatment_probs


{0: 0.5024633841079285,
 1: 0.061821104316049875,
 2: 0.06404798680613984,
 3: 0.061883892356183995,
 4: 0.0626666499231893,
 5: 0.06434936939878359,
 6: 0.06265827818450474,
 8: 0.061222525000104645,
 9: 0.05888680990711556}

In [13]:
# Class weights
alpha = 0.9

In [14]:
def fill_cat_missing(df, cat_cols):
    df = df.copy()
    df[cat_cols] = df[cat_cols].astype("string").fillna("MISSING")
    return df

In [15]:
# ── Deploy: CatBoost uncalibrated trained on full dataset ──────────────────
y_series = pd.Series(y, index=X.index)
all_classes = np.unique(y_series)

# 1. Use explicit column lists instead of inferring from dtypes
X_full = X[categorical_cols + numeric_cols].copy()

# 2. Derive cat/num features from those lists
cat_features = categorical_cols
num_features = numeric_cols

# 3. Apply same missing fill
X_full = fill_cat_missing(X_full, cat_features)

class_labels = np.unique(y_series)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=class_labels,
    y=y_series,
)
class_names = list(class_labels)

cat_model = CatBoostClassifier(
    verbose=100,
    random_seed=42,
    class_names=class_names,
    class_weights=class_weights,
        loss_function="MultiClass",
        thread_count=-1,
)

cat_model.fit(
    X_full,
    y_series,
    cat_features=cat_features,
)

print("Training complete. Class order:", cat_model.classes_)
print("Feature count:", len(cat_model.feature_names_))

Learning rate set to 0.104619
0:	learn: 2.8715006	total: 1.59s	remaining: 26m 25s
10:	learn: 2.7968237	total: 15.8s	remaining: 23m 36s
20:	learn: 2.7523045	total: 29.5s	remaining: 22m 56s
30:	learn: 2.7054974	total: 42.5s	remaining: 22m 9s
40:	learn: 2.6827725	total: 55.1s	remaining: 21m 28s
50:	learn: 2.6621266	total: 1m 5s	remaining: 20m 19s
60:	learn: 2.6466608	total: 1m 17s	remaining: 19m 51s
70:	learn: 2.6104496	total: 1m 31s	remaining: 19m 56s
80:	learn: 2.5884656	total: 1m 44s	remaining: 19m 47s
90:	learn: 2.5702371	total: 1m 57s	remaining: 19m 29s
100:	learn: 2.5599131	total: 2m 7s	remaining: 18m 55s
110:	learn: 2.5366157	total: 2m 17s	remaining: 18m 24s
120:	learn: 2.4981694	total: 2m 33s	remaining: 18m 32s
130:	learn: 2.4613463	total: 2m 47s	remaining: 18m 33s
140:	learn: 2.4191256	total: 3m 3s	remaining: 18m 35s
150:	learn: 2.3766518	total: 3m 17s	remaining: 18m 32s
160:	learn: 2.3330034	total: 3m 32s	remaining: 18m 29s
170:	learn: 2.2889323	total: 3m 48s	remaining: 18m 27s


In [16]:
from sklearn.metrics import classification_report

proba_train = cat_model.predict_proba(X_full)
y_pred_train = cat_model.classes_[np.argmax(proba_train, axis=1)]

print(classification_report(y_series, y_pred_train))

               precision    recall  f1-score   support

   cameback_0       0.04      0.90      0.07      1391
   cameback_1       0.05      1.00      0.09       185
   cameback_2       0.06      1.00      0.11       190
   cameback_3       0.04      1.00      0.07       203
   cameback_4       0.05      1.00      0.09       217
   cameback_5       0.04      1.00      0.08       224
   cameback_6       0.04      1.00      0.08       229
   cameback_8       0.05      1.00      0.09       183
   cameback_9       0.05      1.00      0.10       175
no_cameback_0       0.69      0.03      0.05    118647
no_cameback_1       0.15      0.23      0.18     14584
no_cameback_2       0.15      0.22      0.18     15111
no_cameback_3       0.16      0.21      0.18     14581
no_cameback_4       0.15      0.21      0.18     14754
no_cameback_5       0.16      0.21      0.18     15149
no_cameback_6       0.16      0.21      0.18     14740
no_cameback_8       0.15      0.22      0.18     14443
no_cameba

## Deployement dataset

In [17]:
# Load the full dataset and get the other 50%
file_path_2 = r"Data/covariates_deployment_dataset_2026-02-28.csv"
df_half_2 = pd.read_csv(file_path_2)

# Recreate the first 50% with same random_state to get the exact same indices
df_half_2_shuffled = df_half_2.sample(frac=0.5, random_state=42)

# Get the other 50%
df_deployment = df_half_2.drop(df_half_2_shuffled.index).reset_index(drop=True)

C:\Users\tsterk\AppData\Local\Temp\ipykernel_3912\2742076512.py:3: DtypeWarning: Columns (0: total_volume, 1: food_total, 2: sports_total, 3: beauty_total, 4: monetary_value_52wk, 5: online_sales_52w, 6: retail_sales_52w, 7: monetary_value_53w_104w, 8: online_sales_53w_104w, 9: retail_sales_53w_104w) have mixed types. Specify dtype option on import or set low_memory=False.
  df_half_2 = pd.read_csv(file_path_2)


In [18]:
df_deployment['country_sk'].value_counts()

country_sk
hbi|eu|nl    93056
hbi|eu|be    39844
hbi|eu|uk      322
hbi|eu|ie       47
Name: count, dtype: int64

In [19]:
df_deployment = coerce_metrics_to_numeric(df_deployment, numeric_cols)

df_deployment[categorical_cols] = df_deployment[categorical_cols].astype("object")
df_deployment[numeric_cols] = df_deployment[numeric_cols].astype("int64")


In [20]:
# ── Inference on deployment data ────────────────────────────────────────────
# 1. Keep only the features used during training
df_deploy_processed = df_deployment[categorical_cols + numeric_cols].copy()

# 2. Apply same missing fill as training
df_deploy_processed = fill_cat_missing(df_deploy_processed, cat_features)

# 3. Predict
proba_deploy = cat_model.predict_proba(df_deploy_processed)
y_pred_deploy = cat_model.classes_[np.argmax(proba_deploy, axis=1)]
confidence_deploy = np.max(proba_deploy, axis=1)

# 4. Output dataframe
df_predictions = pd.DataFrame(
    proba_deploy,
    index=df_deployment.index,
    columns=[f"p_{c}" for c in cat_model.classes_],
).assign(
    y_pred=y_pred_deploy,
    confidence=confidence_deploy,
)

print(df_predictions.head())
print("\nPredicted class distribution:\n", df_predictions["y_pred"].value_counts(normalize=True))

   p_cameback_0  p_cameback_1  p_cameback_2  p_cameback_3  p_cameback_4  \
0      0.183114      0.027121      0.067113      0.005156      0.018888   
1      0.006550      0.003008      0.000209      0.000174      0.000271   
2      0.092111      0.010692      0.000164      0.004195      0.000047   
3      0.205982      0.008233      0.012217      0.002987      0.000070   
4      0.106288      0.147137      0.133305      0.035843      0.000417   

   p_cameback_5  p_cameback_6  p_cameback_8  p_cameback_9  p_no_cameback_0  \
0      0.037968      0.009907      0.038261      0.002348         0.060983   
1      0.000055      0.000632      0.001440      0.000036         0.124855   
2      0.002201      0.001965      0.002929      0.001976         0.114612   
3      0.001664      0.017059      0.019991      0.006861         0.103826   
4      0.029601      0.038443      0.049789      0.006369         0.043321   

   p_no_cameback_1  p_no_cameback_2  p_no_cameback_3  p_no_cameback_4  \
0      

In [21]:
k_values = [1, 2, 3, 4,5,6,8,9]

df_preds_uplift = add_uplifts(
    df_predictions,
    k_values=k_values,
    resp_prefix="cameback",
    nonresp_prefix="no_cameback",
    treatment_probs=treatment_probs
)

In [22]:
uplift_cols = ["uplift_1", "uplift_2", "uplift_3", "uplift_4", "uplift_5", "uplift_6", "uplift_8", "uplift_9"]

# Column name of the best treatment per row (e.g., "uplift_3")
best_col = df_preds_uplift[uplift_cols].idxmax(axis=1)

# Optimal uplift value
df_preds_uplift["optimal_lift"] = df_preds_uplift[uplift_cols].max(axis=1)

# Optimal treatment ID (robust to multi-digit)
df_preds_uplift["optimal_treatment"] = (
    best_col
    .str.extract(r"(\d+)$")[0]
    .astype(int)
)

# If you need it in deployment df
df_deployment["Optimal_treatment"] = df_preds_uplift["optimal_treatment"]
df_deployment["Optimal_uplift"] = df_preds_uplift["optimal_lift"]


In [23]:
# Reverse lookup: map experiment_k back to the original test filename
k_to_filename = {
    1: "BNLX_ChurnP_10_test_export.csv",
    2: "BNLX_ChurnP_25_test_export.csv",
    3: "BNLX_ChurnP_5eu_test_export.csv",
    4: "BNLX_ChurnP_10eu_test_export.csv",
    5: "BNLX_ChurnP_250_test_export.csv",
    6: "BNLX_ChurnP_500_test_export.csv",
    7: "BNLX_ChurnP_SKUd_test_export.csv",
    8: "BNLX_ChurnP_SKUe_test_export.csv",
    9: "BNLX_ChurnP_niks_test_export.csv",
}

df_deployment["original_incentive_name"] = df_deployment["Optimal_treatment"].map(k_to_filename)

In [24]:
df_top_30 = (
    df_deployment
    .sort_values(by="Optimal_uplift", ascending=False)
    .head(int(len(df_deployment) * 0.30))
)
df_top_30['Optimal_treatment'].value_counts()

Optimal_treatment
6    6500
3    6209
5    5739
4    5223
8    4311
1    4220
2    4146
9    3632
Name: count, dtype: int64

In [29]:
# Ensure deterministic order (optional but usually important)
df_top_30 = df_top_30.reset_index(drop=True)

test_selected_multi_uplift = df_top_30.iloc[0::2].reset_index(drop=True)    # rows 0,2,4,...
control_selected_multi_uplift = df_top_30.iloc[1::2].reset_index(drop=True) # rows 1,3,5,...


treatment_output_multi = test_selected_multi_uplift[["customer_nk", "original_incentive_name"]].copy()
treatment_output_multi.to_csv("Input_phase_2/treatment_selected_multi.csv", index=False)

# --- Multi Control file ---
control_output_multi = control_selected_multi_uplift[["customer_nk", "original_incentive_name"]].copy()
control_output_multi["original_incentive_name"] = "Control"
control_output_multi.to_csv("Input_phase_2/control_selected_multi.csv", index=False)

print(f"Multi Treatment rows: {len(treatment_output_multi)}")
print(f"Multi Control rows:   {len(control_output_multi)}")

Multi Treatment rows: 19990
Multi Control rows:   19990


In [26]:
test_selected_multi_uplift['original_incentive_name'].value_counts()

original_incentive_name
BNLX_ChurnP_500_test_export.csv     3265
BNLX_ChurnP_5eu_test_export.csv     3130
BNLX_ChurnP_250_test_export.csv     2865
BNLX_ChurnP_10eu_test_export.csv    2613
BNLX_ChurnP_SKUe_test_export.csv    2150
BNLX_ChurnP_10_test_export.csv      2110
BNLX_ChurnP_25_test_export.csv      2072
BNLX_ChurnP_niks_test_export.csv    1785
Name: count, dtype: int64

In [28]:
# Sanity check pattern we expec tto see 5eu lower better, 10eu higher better!
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

def plot_monetary_value_kde_top30(predicted_top30_per_treatment, clip_max=500):
    incentives = ["BNLX_ChurnP_5eu_test_export.csv", "BNLX_ChurnP_10eu_test_export.csv"]

    df_plot = predicted_top30_per_treatment.loc[
        predicted_top30_per_treatment["original_incentive_name"].isin(incentives),
        ["original_incentive_name", "monetary_value"]
    ].dropna(subset=["monetary_value"]).copy()

    df_plot["monetary_value"] = df_plot["monetary_value"].clip(upper=clip_max)

    x_grid = np.linspace(0, clip_max, 500)

    fig = go.Figure()

    for name, group in df_plot.groupby("original_incentive_name"):
        values = group["monetary_value"].values

        if len(values) < 2:
            continue

        kde = gaussian_kde(values)
        density = kde(x_grid)

        fig.add_trace(
            go.Scatter(
                x=x_grid,
                y=density,
                mode="lines",
                fill="tozeroy",
                name=name
            )
        )

    fig.update_layout(
        title=f"Monetary value KDE (clipped at {clip_max}) | 5eur vs 10eu",
        xaxis_title=f"Monetary value (clipped at {clip_max})",
        yaxis_title="Density",
        template="plotly_white"
    )

    fig.update_xaxes(range=[0, clip_max])

    fig.show()

plot_monetary_value_kde_top30(test_selected_multi_uplift, clip_max=500)